In [6]:

import numpy as np
import pandas as pd
import operator


cut_programs = np.load('../data/cut_Programs.npy')
cut_Question = np.load('../data/cut_Questions.npy')

def add_word_dict(w, word_dict):
    if not w in word_dict:
        word_dict[w] = 1
    else:
        word_dict[w] += 1

def RmOutOfVoc(cut_programs, cut_Question):
    word_dict = dict()
    for program in cut_programs:
        for lines in program:
            for line in lines:
                for w in line:
                    add_word_dict(w, word_dict)

    for question in cut_Question:
        lines = question[0]
        for line in lines:
            for w in line:
                add_word_dict(w, word_dict)

        for i in range(1, 7):
            line = question[i]
            for w in line:
                add_word_dict(w, word_dict)
                
    word_dict = sorted(word_dict.items(), key=operator.itemgetter(1), reverse=True)
    VOC_SIZE = 15000
    VOC_START = 25

    voc_dict = word_dict[VOC_START:VOC_START+VOC_SIZE]
    return voc_dict

voc_dict = RmOutOfVoc(cut_programs, cut_Question)
np.save('voc_dict.npy', voc_dict)

print("done")

done


In [7]:
import numpy as np
import pandas as pd

cut_programs = np.load('../data/cut_Programs.npy')                                                                
voc_dict = np.load('./voc_dict.npy')                                                                              
                                                                                                                  
sample_doc = []                                                                                                   
for cut_program in cut_programs:                                                                                  
    for episode in cut_program:                                                                                   
        for line in episode:                                                                                      
            sample_line = ''                                                                                      
            for w in line:                                                                                        
                if w in voc_dict:                                                                                 
                    sample_line += w+' '                                                                          
            sample_doc.append(sample_line)                                                                        
                                                                                                                  
np.save('./sample_doc.npy', sample_doc)    
print("done")

done


In [8]:
import random
import numpy as np
import pandas as pd

cut_programs = np.load('../data/cut_Programs.npy')

NUM_TRAIN = 50000
TRAIN_VALID_RATE = 0.7
NUM_PROGRAM = 8
def generate_training_data():
    Xs, Ys = [], []
    
    for i in range(NUM_TRAIN):
        pos_or_neg = random.randint(0, 1)
        
        if pos_or_neg==1:
            program_id = random.randint(0, NUM_PROGRAM-1)
            episode_id = random.randint(0, len(cut_programs[program_id])-1)
            line_id = random.randint(0, len(cut_programs[program_id][episode_id])-2)
            
            Xs.append([cut_programs[program_id][episode_id][line_id], 
                       cut_programs[program_id][episode_id][line_id+1]])
            Ys.append(1)
            
        else:
            first_program_id = random.randint(0, NUM_PROGRAM-1)
            first_episode_id = random.randint(0, len(cut_programs[first_program_id])-1)
            first_line_id = random.randint(0, len(cut_programs[first_program_id][first_episode_id])-1)
            
            second_program_id = random.randint(0, NUM_PROGRAM-1)
            second_episode_id = random.randint(0, len(cut_programs[second_program_id])-1)
            second_line_id = random.randint(0, len(cut_programs[second_program_id][second_episode_id])-1)
            
            Xs.append([cut_programs[first_program_id][first_episode_id][first_line_id], 
                       cut_programs[second_program_id][second_episode_id][second_line_id]])
            Ys.append(0)
    
    return Xs, Ys

#Xs, Ys = generate_training_data()

#x_train, y_train = Xs[:int(NUM_TRAIN*TRAIN_VALID_RATE)], Ys[:int(NUM_TRAIN*TRAIN_VALID_RATE)]
#x_valid, y_valid = Xs[int(NUM_TRAIN*TRAIN_VALID_RATE):], Ys[int(NUM_TRAIN*TRAIN_VALID_RATE):]
print("done")

done


In [9]:
from sklearn.feature_extraction.text import HashingVectorizer
import numpy as np
import pandas as pd
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import roc_auc_score
import csv

#sample_doc = np.load('./sample_doc.npy')
Xs, Ys = generate_training_data()

X_vec = []

for X in Xs:
    X[0].extend(X[1])
    X_vec.append(X[0])
Xs = X_vec

with open('random.csv', 'w', newline='') as csvfile:
    fieldnames = ['sentence', 'relation']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    

for x, y in zip(Xs, Ys):
    with open('random.csv', 'a', newline='') as csvfile:
        fieldnames = ['sentence', 'relation']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writerow({'sentence': x, 'relation': y})

print("done")

done


In [10]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import roc_auc_score
import pandas as pd

def get_stream(path, size):
    for chunk in pd.read_csv(path, chunksize=size):
        yield chunk



hashvec = HashingVectorizer(n_features=2**10)
# loss='log' gives logistic regression
clf = SGDClassifier(loss='log', n_iter=100)

batch_size = 2000
stream = get_stream(path='random.csv', size=batch_size)

classes = np.array([0, 1])
train_auc, val_auc = [], []

# we use one batch for training and another for validation in each iteration
iters = int((NUM_TRAIN+batch_size-1)/(batch_size*2))

for i in range(iters):
    batch = next(stream)
    X_train, y_train = batch['sentence'], batch['relation']
    if X_train is None:
        break
        
    X_train = hashvec.transform(X_train)
    clf.partial_fit(X_train, y_train, classes=classes)
    train_auc.append(roc_auc_score(y_train, clf.predict_proba(X_train)[:,1]))
    
    # validate
    batch = next(stream)
    X_val, y_val = batch['sentence'], batch['relation']
    score = roc_auc_score(y_val, clf.predict_proba(hashvec.transform(X_val))[:,1])
    val_auc.append(score)
    print('[%d/%d] %f' % ((i+1)*(batch_size*2), NUM_TRAIN, score))

print("done")

[4000/50000] 0.514282


/home/jrchang/workspace/anaconda/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
/home/jrchang/workspace/anaconda/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
/home/jrchang/workspace/anaconda/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
/home/jrchang/workspace/anaconda/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)


[8000/50000] 0.513656
[12000/50000] 0.473250
[16000/50000] 0.526044
[20000/50000] 0.484597
[24000/50000] 0.498456
[28000/50000] 0.509048
[32000/50000] 0.507303
[36000/50000] 0.498136
[40000/50000] 0.497882
[44000/50000] 0.497702
[48000/50000] 0.510830
done


/home/jrchang/workspace/anaconda/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
/home/jrchang/workspace/anaconda/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
/home/jrchang/workspace/anaconda/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
/home/jrchang/workspace/anaconda/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:117: DeprecationWarning: n_iter parameter is deprecated in 0.19 and will be removed in 0.21. Use max_iter and tol instead.
  DeprecationWarning)
